In [329]:
import cv2
import numpy as np
import json
import os
from numba import jit
import struct

In [330]:
def generate_RD(gray,rangeSize,domainSize,stepSize):
    # 將圖片拆成4x4的Range Block
    range_Blocks=np.empty((gray.shape[0]//rangeSize,gray.shape[1]//rangeSize,rangeSize,rangeSize),np.uint8)
    for row in range(gray.shape[0]//rangeSize):
        for col in range(gray.shape[1]//rangeSize):
            range_Blocks[row,col]=gray[row*rangeSize:(row+1)*rangeSize,col*rangeSize:(col+1)*rangeSize].copy()

    # 將圖片拆成8x8的Domain Block
    domain_Blocks=np.empty(((gray.shape[0]-domainSize)//stepSize+1,(gray.shape[1]-domainSize)//stepSize+1,domainSize,domainSize),np.uint8)
    for row in range(domain_Blocks.shape[0]):
        for col in range(domain_Blocks.shape[1]):
            domain_Blocks[row,col]=gray[row*stepSize:row*stepSize+domainSize,col*stepSize:col*stepSize+domainSize].copy()

    # Domain Block均值收縮成Range Block大小
    domainContraction_Blocks=np.empty((domain_Blocks.shape[0],domain_Blocks.shape[1],rangeSize,rangeSize),np.uint8)
    for row in range(domain_Blocks.shape[0]):
        for col in range(domain_Blocks.shape[1]):
            domainContraction_Blocks[row,col]=cv2.resize(domain_Blocks[row,col],(rangeSize,rangeSize))

    return range_Blocks,domainContraction_Blocks

In [331]:
@jit
def cal_SO(R_block,D_block,rangeSize):
    meanR=np.sum(R_block)/(rangeSize**2)
    meanD=np.sum(D_block)/(rangeSize**2)

    s=abs(np.sum((R_block-meanR)*(D_block-meanD))/np.sum((D_block-meanD)**2))
    o=int(abs((meanR-s*meanD)/255*128))

    if s>1:
        s=0
    elif s<0.25:
        s=0.25
    elif s<0.5:
        s=0.5
    elif s<0.75:
        s=0.75
    else:
        s=1

    if o>128:
        o=128

    return s,o

C:\Users\jush\AppData\Local\Temp\ipykernel_16564\1519318730.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def cal_SO(R_block,D_block,rangeSize):


In [332]:
@jit
def cal_Difference(R,D,s,o):
    diff = R-(D*s+o)
    for row in range(len(diff)):
        for col in range(len(diff[row])):
            diff[row][col]=abs(diff[row][col])
    return np.sum(diff)

C:\Users\jush\AppData\Local\Temp\ipykernel_16564\1113700695.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def cal_Difference(R,D,s,o):


In [333]:
@jit
def transform(D_block,method):
    match method:
        case 0:
            # 無轉換
            return D_block
        case 1:
            # 左右翻轉
            return np.fliplr(D_block)
        case 2:
            # 上下翻轉
            return np.flipud(D_block)
        case 3:
            # 主對角對稱
            return D_block.T
        case 4:
            # 次對角對稱
            return np.flip(D_block.T)
        case 5:
            # 旋轉90度
            return np.rot90(D_block,1)
        case 6:
            # 旋轉180度
            return np.rot90(D_block,2)
        case 7:
            # 旋轉270度
            return np.rot90(D_block,3)
        case _:
            return D_block

C:\Users\jush\AppData\Local\Temp\ipykernel_16564\41874990.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def transform(D_block,method):


In [334]:
@jit
def find_min(R_block,D_block,rangeSize):
    s,o=cal_SO(R_block,D_block,rangeSize)

    D=transform(D_block,0)
    diff=cal_Difference(R_block,D,s,o)
    min_diff= diff
    method=0

    for i in range(1,8):
        D=transform(D_block,i)
        diff=cal_Difference(R_block,D,s,o)
        if  diff < min_diff:
            min_diff = diff
            method=i

    return method,min_diff,s,o

C:\Users\jush\AppData\Local\Temp\ipykernel_16564\1259411739.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def find_min(R_block,D_block,rangeSize):


In [335]:
@jit
def minBlock(range_Blocks,domainContraction_Blocks,rangeSize):
    min_diff= float(1e9)
    row,col,method,s,o=0,0,0,0,0
    for row_d in range(domainContraction_Blocks.shape[0]):
        for col_d in range(domainContraction_Blocks.shape[1]):
            method_,diff,s_,o_=find_min(range_Blocks,domainContraction_Blocks[row_d,col_d],rangeSize)
            if diff< min_diff:
                min_diff=diff
                row,col,method,s,o=row_d,col_d,method_,s_,o_
    return row,col,method,s,o

C:\Users\jush\AppData\Local\Temp\ipykernel_16564\405888988.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def minBlock(range_Blocks,domainContraction_Blocks,rangeSize):


In [398]:
def compress(file,rangeSize,domainSize,stepSize):
    # 壓縮 輸出min_block = D索引(2維) , 轉換方法(0~7)  , s(0,0.25,0.5,0.75,1) , o(0~128)
    min_block=[]
    # 使用cv2.imread讀取是BGR格式
    img=cv2.imread(file)
    # 轉成灰階圖
    img=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # min_block[0] = 高 , 寬 , rangeSize , domainSize , stepSize
    min_block.append([img.shape[0],img.shape[1],rangeSize,domainSize,stepSize])
    byte_data=bytes([img.shape[0],img.shape[1],rangeSize,domainSize,stepSize])
    # bytes_str = struct.pack('2i3c',img.shape[0],img.shape[1], b''.join(rangeSize),b''.join(domainSize), b''.join(stepSize))
    range_Blocks,domainContraction_Blocks=generate_RD(img,rangeSize,domainSize,stepSize)
    for row in range(range_Blocks.shape[0]):
        min_block.append([])
        # byte_data+=b'{'
        for col in range(range_Blocks.shape[1]):
            print("R:",row,col)
            R_block=range_Blocks[row,col]
            r,c,method,s,o=minBlock(R_block,domainContraction_Blocks,rangeSize)
            min_block[-1].append([r,c,method,int(s*4),o])
            byte_data+=bytes([r,c,method,int(s*4),o])


    name_without_extension = os.path.splitext(os.path.basename(file))[0]
    # 將壓縮結果輸出成json檔
    # with open("output/"+name_without_extension+".json", 'w') as file:
    #     json.dump(min_block, file)

    with open("output/"+name_without_extension+".bin", 'wb') as file:
        # 將每個 bytes 資料寫入檔案
        file.write(byte_data)

    # return min_block

In [399]:
compress_output=compress("input-Lenna-2.jpg",2,4,2)

R: 0 0
R: 0 1
R: 0 2
R: 0 3
R: 0 4
R: 0 5
R: 0 6
R: 0 7
R: 0 8
R: 0 9
R: 0 10
R: 0 11
R: 0 12
R: 0 13
R: 0 14
R: 0 15
R: 0 16
R: 0 17
R: 0 18
R: 0 19
R: 0 20
R: 0 21
R: 0 22
R: 0 23
R: 0 24
R: 0 25
R: 0 26
R: 0 27
R: 0 28
R: 0 29
R: 0 30
R: 0 31
R: 0 32
R: 0 33
R: 0 34
R: 0 35
R: 0 36
R: 0 37
R: 0 38
R: 0 39
R: 0 40
R: 0 41
R: 0 42
R: 0 43
R: 0 44
R: 0 45
R: 0 46
R: 0 47
R: 0 48
R: 0 49
R: 0 50
R: 0 51
R: 0 52
R: 0 53
R: 0 54
R: 0 55
R: 0 56
R: 0 57
R: 0 58
R: 0 59
R: 0 60
R: 0 61
R: 0 62
R: 0 63
R: 1 0
R: 1 1
R: 1 2
R: 1 3
R: 1 4
R: 1 5
R: 1 6
R: 1 7
R: 1 8
R: 1 9
R: 1 10
R: 1 11
R: 1 12
R: 1 13
R: 1 14
R: 1 15
R: 1 16
R: 1 17
R: 1 18
R: 1 19
R: 1 20
R: 1 21
R: 1 22
R: 1 23
R: 1 24
R: 1 25
R: 1 26
R: 1 27
R: 1 28
R: 1 29
R: 1 30
R: 1 31
R: 1 32
R: 1 33
R: 1 34
R: 1 35
R: 1 36
R: 1 37
R: 1 38
R: 1 39
R: 1 40
R: 1 41
R: 1 42
R: 1 43
R: 1 44
R: 1 45
R: 1 46
R: 1 47
R: 1 48
R: 1 49
R: 1 50
R: 1 51
R: 1 52
R: 1 53
R: 1 54
R: 1 55
R: 1 56
R: 1 57
R: 1 58
R: 1 59
R: 1 60
R: 1 61
R: 1 62
R: 1